In [1]:
from __future__ import division
from pyomo.opt import SolverFactory
from pyomo.environ import *
import time
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import norm
import seaborn as sns
from pyomo.opt import SolverStatus, TerminationCondition

In [2]:
# Assume everyday the departure time and arrival time follows normal
# distribution (8:00, -10:00) departure time (4:00- 6:00)

#generate start time and end time for 100 EVs
# Truncated normal distribution 
lower_arr, upper_arr = 24, 48
mu_arr, sigma_arr = 36, 4
samples_arr = stats.truncnorm.rvs(
          (lower_arr-mu_arr)/sigma_arr,(upper_arr-mu_arr)/sigma_arr,loc=mu_arr,scale=sigma_arr,size=100)
ArrTime = np.round(samples_arr)


lower_dep, upper_dep = 56, 80
mu_dep, sigma_dep = 68, 4
samples_dep = stats.truncnorm.rvs(
          (lower_dep-mu_dep)/sigma_dep,(upper_dep-mu_dep)/sigma_dep,loc=mu_dep,scale=sigma_dep,size=100)
DepTime = np.round(samples_dep)

# # uniform distribution
# ArrTime = [round(i) for i in np.random.normal(31,41,100)]
# DepTime = [round(i) for i in np.random.normal(63,73,100)]

TimeTable = pd.DataFrame({'Arrival time': ArrTime, 'Departure time': DepTime})

# Create the 0-1 table to show the plug-in time
PlugTime = [[0 for i in range(96)] for j in range(100)]
for x in range(100):
    index = 0
    while index < 96:
        if index > TimeTable['Arrival time'].iloc[x] and index < TimeTable['Departure time'].iloc[x]:
            PlugTime[x][index] = 1
        else:
            PlugTime[x][index] = 0
        index += 1

PlugTimeTable = pd.DataFrame(PlugTime)

In [3]:
# calculate energy consumption of each EV
lower_con, upper_con = 8, 80
mu_con, sigma_con = 20, 8
samples_con = stats.truncnorm.rvs(
          (lower_con-mu_con)/sigma_con,(upper_con-mu_con)/sigma_con,loc=mu_con,scale=sigma_con,size=100)
energy_consump = {k: v for k, v in enumerate(samples_con/3.75)}

In [ ]:
# #calculate uncontroll charging load
# energy_consump = np.random.uniform(5/3.75, 40/3.75, 50)

charge_activity = [[0 for i in range(96)] for j in range(100)]
cum_energy = 0
for i in range(100):
    for j in range(96):
        if PlugTimeTable.iloc[i][j] == 1 and energy_consump[i] - cum_energy >= 6.6/4:
            charge_activity[i][j] = 6.6
            cum_energy += 6.6/4
            
        elif PlugTimeTable.iloc[i][j] == 1 and cum_energy < energy_consump[i] and energy_consump[i] - cum_energy <6.6/4:
            charge_activity[i][j] = (energy_consump[i] - cum_energy) * 4
            cum_energy = 0
            break
        else:
            charge_activity[i][j] = 0
    
EV_Load = pd.DataFrame(charge_activity).sum(axis = 0)
EV_Load.plot()
plt.xlabel("Time") 
plt.ylabel("load")
plt.show()

In [5]:
# Load the load and price data
opt_input = pd.read_excel('data.xlsx', sheetname=0, header=0, index_col=None)
Building_load = opt_input['Demand (kW)']

#Building_load.to_dict()

# total_load = EV_Load + Building_load
# total_load.plot()
# Building_load.plot()
# plt.xlabel("Time") 
# plt.ylabel("load")
# plt.show()

In [6]:
def Co_optimization(inputfile, TimeTable, energydemand, solver="gurobi"):
    # Read all price data

    price = inputfile['Energy Price ($/kWh)'].to_dict()
    netload = inputfile['Demand (kW)'].to_dict()
    


    with SolverFactory(solver) as opt:
        # Creation of a Concrete Model
            model = ConcreteModel()


            # ###### Set
            model.t = Set(initialize=range(96), doc = 'Time')
            model.v = Set(initialize=range(100), doc='Vehicles')

            # ###### Parameters

            # Price, load, required energy
            model.price_energy = Param(model.t, initialize=price, doc='Energy Price')
            model.net_load = Param(model.t, initialize = netload, doc = 'net load')
            model.energy_demand = Param(model.v, initialize = energydemand, doc = 'required energy')


            # ###### Variable
            model.energy_charge = Var(model.t, model.v, domain=NonNegativeReals, doc='charge power')
            model.D = Var(domain = NonNegativeReals, doc = 'added demand charge cost')


            # ###### Rules
            def maximum_chargepower_rule(model, t, v):
                return model.energy_charge[t,v] <= 6.6
            model.chargepower_max_rule1 = Constraint(model.t, model.v, rule=maximum_chargepower_rule, doc='Pcharge max rule')


            def final_energy_balance(model, v):
                return sum(model.energy_charge[i,v] * TimeTable[i][v] for i in model.t)/4 >= model.energy_demand[v]
            model.final_energy_rule = Constraint(model.v, rule=final_energy_balance, doc='E final rule')


            def demand_charge_rule(model, t):
                return 18.74 * (sum(model.energy_charge[t,v] for v in model.v) + model.net_load[t]) <= model.D
            model.demand_charge_rule = Constraint(model.t, rule=demand_charge_rule, doc='cannot exceed demand charge')

            def objective_rule(model):
                return sum(model.price_energy[t] * (sum(model.energy_charge[t,v]  for v in model.v) +  model.net_load[t])/4 
                           for t in model.t) * 30 + model.D
            model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')
            

            results = opt.solve(model)
#             result_charge = []


#             for i in model.energy_charge:
#                 result_charge.append(value(model.energy_charge[i]))
#             optimalresult = pd.DataFrame(
#                 {'energy charge' : result_charge}
#             )


#             print value(model.objective)
#             print value(model.D)
#             model.load(results) # Loading solution into results object

#             if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
#                 print 'Got the optimal solution!'
#             elif (results.solver.termination_condition == TerminationCondition.infeasible):
#                 print 'No infeasible solution!'
#             else:
#                 # Something else is wrong
#                 print ('Solver Status:', results.solver.status)


    return model, results

In [7]:
# # Optimization to minimize energy cost and demand charge
myresult = Co_optimization(opt_input, PlugTimeTable, energy_consump)

myresult

(<pyomo.core.base.PyomoModel.ConcreteModel at 0x1a1866dc80>,
 {'Problem': [{'Number of objectives': 1, 'Lower bound': 96812.805139, 'Name': '', 'Number of variables': 9602, 'Upper bound': 96812.805139, 'Number of continuous variables': 9602, 'Sense': 'minimize', 'Number of constraints': 9797, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of nonzeros': 22361}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Wall time': '0.0136029720306', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Time': 0.19733691215515137, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Error rc': 0, 'Return code': '0'}]})